In [20]:
### Sample First Rate Data
### https://firstratedata.com/free-intraday-data

import pandas as pd
from datetime import datetime, date, timedelta
import warnings
import numpy as np
import h5py
import os
from multiprocessing import Pool, Lock
# os.environ['PYDEVD_DISABLE_FILE_VALIDATION'] = 1
# os.environ['PYTHONNOUSERSITE'] = 1

# Filter out all warnings
warnings.filterwarnings("ignore")

# Specify the directory path
input_directory = 'sample_data'
output_directory = 'outputs'

# Get list of files in the directory
files = os.listdir(input_directory)

input_files_dict ={}

# Loop through files in the directory
for file in os.listdir(input_directory):
    # Check if the file ends with .csv or .txt
    if file.endswith('.csv') or file.endswith('.txt'):
        # Extract the key from the file name until the first "_" character
        key = file.split('_')[0]
       # Read the CSV or TXT file into a Pandas DataFrame
        file_path = os.path.join(input_directory, file)
        if file.endswith('.txt'):
            input_files_dict[key] = pd.read_csv(file_path, header = None) ##if txt file, no headers
        else:
            input_files_dict[key] = pd.read_csv(file_path)
# Print the dictionary containing file names and df data types
for key, df in input_files_dict.items():
    print(f"Key: {key}")
    
for key, df in input_files_dict.items():
    print(f"Key: {key}")
    print(df.dtypes)

Key: AAPL
Key: AMZN
Key: DIA
Key: DJI
Key: EEM
Key: META
Key: MSFT
Key: NDX
Key: QQQ
Key: RUT
Key: SPX
Key: SPY
Key: TSLA
Key: VIX
Key: VXX
Key: AAPL
timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object
Key: AMZN
timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object
Key: DIA
timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object
Key: DJI
0     object
1    float64
2    float64
3    float64
4    float64
dtype: object
Key: EEM
timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object
Key: META
timestamp     object
open         float64
high         float64
low          float64
close        float64
volume         int64
dtype: object
Key: MSFT
timestamp     

In [21]:
#Add headers for indices and stocks/ETFs that need it
dfs_to_remove = []
for key, df in input_files_dict.items():
  if df.columns[0] != 'timestamp' and df.shape[1] == 5:
    df.columns = ['timestamp', 'open', 'high', 'low', 'close']
    df['open']=df['open'].astype('float64')
    df['high']=df['high'].astype('float64')
    df['low']=df['low'].astype('float64')
    df['close']=df['close'].astype('float64')
  elif df.columns[0] != 'timestamp' and df.shape[1] == 6:
    df.columns = ['timestamp', 'open', 'high', 'low', 'close','volume']
    df['open']=df['open'].astype('float64')
    df['high']=df['high'].astype('float64')
    df['low']=df['low'].astype('float64')
    df['close']=df['close'].astype('float64')
    df['volume']=df['volume'].astype('int64')
  else:
    pass


In [22]:
#change timestamp to datetime data type
for key, df in input_files_dict.items():
  df['timestamp'] = pd.to_datetime(df['timestamp'])
  print(df.dtypes)

#add time column
for key, df in input_files_dict.items():
  df['time'] = df['timestamp'].dt.time
  print(key)
  print(df.head(10))


timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
dtype: object
timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
dtype: object
timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
dtype: object
timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
dtype: object
timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close               float64
volume                int64
dtype: object
timestamp    datetime64[ns]
open                float64
high                float64
low                 float64
close 

In [23]:
# drop pre- and post-market
mkt_open = pd.to_datetime('09:30:00').time()
mkt_close = pd.to_datetime('16:00:00').time()
list_pos = 0
for key, df in input_files_dict.items():
  print(key, df.shape)
  input_files_dict[key] = df.loc[(df['time'] >= mkt_open) & (df['time'] <= mkt_close)].copy()
  print(key, input_files_dict[key].shape)

AAPL (193829, 7)
AAPL (98000, 7)
AMZN (193619, 7)
AMZN (98019, 7)
DIA (119715, 7)
DIA (97614, 7)
DJI (99069, 6)
DJI (97823, 6)
EEM (113256, 7)
EEM (97770, 7)
META (162279, 7)
META (97873, 7)
MSFT (158999, 7)
MSFT (97869, 7)
NDX (97933, 6)
NDX (97933, 6)
QQQ (210482, 7)
QQQ (97953, 7)
RUT (100058, 6)
RUT (97817, 6)
SPX (99074, 6)
SPX (97822, 6)
SPY (207824, 7)
SPY (97994, 7)
TSLA (230816, 7)
TSLA (98112, 7)
VIX (191737, 6)
VIX (97524, 6)
VXX (121604, 7)
VXX (97383, 7)


In [25]:
## Build base database

## generate list of unique dates and order from oldest to newest
date_list = []
for key, df in input_files_dict.items():
  for i in df['timestamp']:
    i = pd.to_datetime(i).date()
    date_list.append(i)
print(len(date_list))

date_list = list(set(date_list))
date_list.sort()
print(date_list[:10])
print(date_list[-10:])
print(len(date_list))

###Add minutes for each day
duration = 391 #trading minutes in a day
datetime_list = []
for i in date_list:
  for k in range(int(duration)):
    x = datetime.combine(i, mkt_open) + timedelta(minutes=(k))
    datetime_list.append(x)

print(datetime_list[:10])
print(len(datetime_list))

df_1 = pd.DataFrame(datetime_list)
df_1.columns = ['timestamp']
print(df_1.head(10))
# df_1.to_csv('/out_csv')

###Add date abnd time each day
df_1['date'] = df_1['timestamp'].dt.date
df_1['time'] = df_1['timestamp'].dt.time

###Add minute markers for each day
df_1['minute'] = 0
min = 1
for i in range(len(df_1['timestamp'])):
  df_1['minute'][i] = min
  if min == 391:
    min = 1
  else:
    min += 1
min = 0


###Add time context for each day
tc_1 = 5
tc_2 = 15
tc_3 = 30
tc_4 = 60
tc_5 = 120
tc_6 = 270
tc_7 = 330
tc_8 = 360
tc_9 = 375
tc_10 = 385
tc_11 = 391

time_context_list = []
for i in df_1['minute']:
  if i <= tc_1:
    time_context = 1
  elif i <= tc_2:
    time_context = 2
  elif i <= tc_3:
    time_context = 3
  elif i <= tc_4:
    time_context = 4
  elif i <= tc_5:
    time_context = 5
  elif i <= tc_6:
    time_context = 6
  elif i <= tc_7:
    time_context = 7
  elif i <= tc_8:
    time_context = 8
  elif i <= tc_9:
    time_context = 9
  elif i <= tc_10:
    time_context = 10
  else:
    time_context = 11
  time_context_list.append(time_context)
df_1['time_context'] = time_context_list

print(df_1.head(20))

1467506
[datetime.date(2022, 9, 30), datetime.date(2022, 10, 3), datetime.date(2022, 10, 4), datetime.date(2022, 10, 5), datetime.date(2022, 10, 6), datetime.date(2022, 10, 7), datetime.date(2022, 10, 10), datetime.date(2022, 10, 11), datetime.date(2022, 10, 12), datetime.date(2022, 10, 13)]
[datetime.date(2023, 9, 18), datetime.date(2023, 9, 19), datetime.date(2023, 9, 20), datetime.date(2023, 9, 21), datetime.date(2023, 9, 22), datetime.date(2023, 9, 25), datetime.date(2023, 9, 26), datetime.date(2023, 9, 27), datetime.date(2023, 9, 28), datetime.date(2023, 9, 29)]
251
[datetime.datetime(2022, 9, 30, 9, 30), datetime.datetime(2022, 9, 30, 9, 31), datetime.datetime(2022, 9, 30, 9, 32), datetime.datetime(2022, 9, 30, 9, 33), datetime.datetime(2022, 9, 30, 9, 34), datetime.datetime(2022, 9, 30, 9, 35), datetime.datetime(2022, 9, 30, 9, 36), datetime.datetime(2022, 9, 30, 9, 37), datetime.datetime(2022, 9, 30, 9, 38), datetime.datetime(2022, 9, 30, 9, 39)]
98141
            timestamp
0 2

In [26]:
#drop time column
for key, df in input_files_dict.items():
  input_files_dict[key] = df.drop(columns=['time']).copy()
  print(key, df.head(10))

#rename other database column headers
for key, df in input_files_dict.items():
  new_col_list = []
  for i in df.columns:
    if i == 'timestamp':
      new_col_name = i
    else:
      new_col_name = key + '_' + i
    new_col_list.append(new_col_name)
  input_files_dict[key].columns=new_col_list

for key, df in input_files_dict.items():
  print(df.head(10))


AAPL               timestamp      open      high     low     close   volume  \
271 2022-09-30 09:30:00  141.2100  141.6850  140.95  141.6808  1553146   
272 2022-09-30 09:31:00  141.6700  142.0650  141.62  142.0321   475394   
273 2022-09-30 09:32:00  142.0350  142.3856  142.02  142.2274   456415   
274 2022-09-30 09:33:00  142.2100  142.4900  142.15  142.4735   337576   
275 2022-09-30 09:34:00  142.4800  142.6000  142.20  142.2900   426136   
276 2022-09-30 09:35:00  142.2800  142.3300  141.90  141.9101   389357   
277 2022-09-30 09:36:00  141.9100  141.9600  141.52  141.5200   399922   
278 2022-09-30 09:37:00  141.5367  141.5400  141.09  141.1575   438725   
279 2022-09-30 09:38:00  141.1550  141.2000  140.78  140.8400   595640   
280 2022-09-30 09:39:00  140.8499  140.9000  140.62  140.8600   483574   

         time  
271  09:30:00  
272  09:31:00  
273  09:32:00  
274  09:33:00  
275  09:34:00  
276  09:35:00  
277  09:36:00  
278  09:37:00  
279  09:38:00  
280  09:39:00  
AMZN

In [27]:
### create dict_2 with new dataframes that have full timestamp column and confirm 5222 blanks
input_files_dict_2 = {}
for key, df in input_files_dict.items():
  merged_df = pd.merge(df_1, df, on='timestamp', how='outer').copy()
  input_files_dict_2[key] = merged_df



counter = 0
for key, df in input_files_dict_2.items(): # each df
  for i in df.columns[5:]: #column in each df excl. timestamp, minute, and time_contect
    for k in range(len(df[i])):
      if pd.isnull(df[i][k]) != True and pd.isna(df[i][k]) != True:
        pass
      else:
        counter += 1
print(counter) ### should equal 5222


#fill in blanks for each data frame
counter_1 = 0 
for key, df in input_files_dict_2.items(): # each df
#create last close dictionary and fill in with initial values
  last_close_dict = {}
  last_close_dict[key+'_last_close'] = 0
  print(last_close_dict)
  for k in range(len(df)):
    if (pd.isnull(df[key+'_close'][k]) == True or pd.isna(df[key+'_close'][k]) == True) and k == 0:
      print('BSH Error: Missing data in first row of dataframe')
    else:
      last_close_dict[key+'_last_close'] = df[key+'_close'][k]
      print(key, last_close_dict[key+'_last_close'])
      break

  counter_2 = 0
  for k in df.columns[5:]:
    print(k)
    for j in range(len(df)):
      if pd.isnull(df[k][j]) != True and pd.isna(df[k][j]) != True:
        last_close_dict[key+'_last_close'] = df[key+'_close'][j]
      else:
        counter_2 += 1
        if 'volume' in k:
          df[k][j] = 0
        else:
          df[k][j] = last_close_dict[key+'_last_close']

  counter_1 = counter_1 + counter_2
print(counter_1)

for key, df in input_files_dict_2.items():
  for i in df:
    print(len(df[i]))

21259
{'AAPL_last_close': 0}
AAPL 141.6808
AAPL_open
AAPL_high
AAPL_low
AAPL_close
AAPL_volume
{'AMZN_last_close': 0}
AMZN 114.03
AMZN_open
AMZN_high
AMZN_low
AMZN_close
AMZN_volume
{'DIA_last_close': 0}
DIA 291.69
DIA_open
DIA_high
DIA_low
DIA_close
DIA_volume
{'DJI_last_close': 0}
DJI 29181.47
DJI_open
DJI_high
DJI_low
DJI_close
{'EEM_last_close': 0}
EEM 34.98
EEM_open
EEM_high
EEM_low
EEM_close
EEM_volume
{'META_last_close': 0}
META 136.96
META_open
META_high
META_low
META_close
META_volume
{'MSFT_last_close': 0}
MSFT 238.1695
MSFT_open
MSFT_high
MSFT_low
MSFT_close
MSFT_volume
{'NDX_last_close': 0}
NDX 11138.4
NDX_open
NDX_high
NDX_low
NDX_close
{'QQQ_last_close': 0}
QQQ 271.43
QQQ_open
QQQ_high
QQQ_low
QQQ_close
QQQ_volume
{'RUT_last_close': 0}
RUT 1671.913
RUT_open
RUT_high
RUT_low
RUT_close
{'SPX_last_close': 0}
SPX 3636.48
SPX_open
SPX_high
SPX_low
SPX_close
{'SPY_last_close': 0}
SPY 362.53
SPY_open
SPY_high
SPY_low
SPY_close
SPY_volume
{'TSLA_last_close': 0}
TSLA 265.3
TSLA_op

In [64]:

#create two lists, one for tickers with volume and a list for indices
dict_vol_dfs = {}
dict_no_vol_dfs = {}
for key, df in input_files_dict_2.items():
    if 'volume' in df.columns[-1]:
        dict_vol_dfs[key] = df
    else:
        dict_no_vol_dfs[key] = df

print('dict_vol_dfs:')
for key, df in dict_vol_dfs.items():
    print(key)

for key, df in dict_vol_dfs.items():
    print(df.head(10))

print('dict_no_vol_dfs:')
for key, df in dict_no_vol_dfs.items():
    print(key)

for key, df in dict_no_vol_dfs.items():
    print(df.head(10))


dict_vol_dfs:
AAPL
AMZN
DIA
EEM
META
MSFT
QQQ
SPY
TSLA
VXX
            timestamp        date      time  minute  time_context  AAPL_open  \
0 2022-09-30 09:30:00  2022-09-30  09:30:00       1             1   141.2100   
1 2022-09-30 09:31:00  2022-09-30  09:31:00       2             1   141.6700   
2 2022-09-30 09:32:00  2022-09-30  09:32:00       3             1   142.0350   
3 2022-09-30 09:33:00  2022-09-30  09:33:00       4             1   142.2100   
4 2022-09-30 09:34:00  2022-09-30  09:34:00       5             1   142.4800   
5 2022-09-30 09:35:00  2022-09-30  09:35:00       6             2   142.2800   
6 2022-09-30 09:36:00  2022-09-30  09:36:00       7             2   141.9100   
7 2022-09-30 09:37:00  2022-09-30  09:37:00       8             2   141.5367   
8 2022-09-30 09:38:00  2022-09-30  09:38:00       9             2   141.1550   
9 2022-09-30 09:39:00  2022-09-30  09:39:00      10             2   140.8499   

   AAPL_high  AAPL_low  AAPL_close  AAPL_volume  
0   141.68

In [65]:
data_frames = [df.set_index('timestamp') for df in dict_vol_dfs.values()]
combined_df = pd.concat(data_frames, axis=1)
# Drop duplicate columns
combined_df = combined_df.loc[:,~combined_df.columns.duplicated()]
# Print the combined DataFrame to check the structure
combined_df.keys()

# Save to CSV
combined_df.to_csv('dict_vol_df_combined.csv', index=False)

In [16]:
### dict_vol_dfs feature creation 

#Begin adding features to data set
SMA_len_list = [10, 30, 60, 120, 180] #also used for vwap
EMA_len_list = [5, 10, 15, 20, 30, 60]
EMA_smooth_list = [1, 1.5, 2, 2.5, 3]


#hlc3, candle size
for key, df in dict_vol_dfs.items():
    print(key + '_hlc3')
    df[key + '_hlc3'] = (df[key + '_high'] + df[key + '_low'] + df[key + '_close'])/3
    print(key + '_candle_size')
    df[key + '_candle_size'] = df[key + '_high'] - df[key + '_low']

#candle color
def compute_candle_color(args):
    key, df = args
    candle_color_list = []
    for k in range(len(df[key + '_hlc3'])):
        if df[key + '_close'][k] > df[key + '_open'][k]:
            candle_color = 1
        elif df[key + '_close'][k] < df[key + '_open'][k]:
            candle_color = 2
        else:
            candle_color = 0
        candle_color_list.append(candle_color)
    return key, candle_color_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_candle_color, [(key, df) for key, df in dict_vol_dfs.items()])

    for key, candle_color_list in results:
        dict_vol_dfs[key][f'{key}_candle_color'] = candle_color_list
        print(key + '_candle_color')


# #SMAs_old
# for key, df in dict_vol_dfs.items():
#     for i in SMA_len_list:
#         print(key + '_SMA_' + str(i))
#         df[key + '_SMA_' + str(i)] = 0
#         for k in range(len(df[key + '_hlc3'])):
#             df[key + '_SMA_' + str(i)][k] = (df[key + '_hlc3'][k-i+1:k+1].sum())/i

#SMAs_new
def compute_SMA(args):
    key, df, i = args
    SMA_list = []
    for k in range(len(df[key + '_hlc3'])):
        SMA = (df[key + '_hlc3'][k-i+1:k+1].sum())/i
        SMA_list.append(SMA)
    return key, i, SMA_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_SMA, [(key, df, i) for key, df in dict_vol_dfs.items() for i in SMA_len_list])

    for key, i, SMA_list in results:
        dict_vol_dfs[key][f'{key}_SMA_{i}'] = SMA_list
        print(key + '_SMA_' + str(i))

# #EMAs_old
# for key, df in dict_vol_dfs.items():  
#     for i in SMA_len_list:
#         for k in EMA_mult_list:
#             print(key + '_EMA_' + str(i) + '_' + str(k))
#             df[key + '_EMA_' + str(i) + '_' + str(k)] = 0
#             for j in range(len(df[key + '_hlc3'])):
#                 if df[key + '_SMA_' + str(i)][j] == 0:
#                     df[key + '_EMA_' + str(i) + '_' + str(k)][j] = 0
#                 elif df[key + '_SMA_' + str(i)][j] != 0 and df[key + '_SMA_' + str(i)][j-1] == 0:
#                     df[key + '_EMA_' + str(i) + '_' + str(k)][j] = df[key + '_SMA_' + str(i)][j]
#                 else:
#                     df[key + '_EMA_' + str(i) + '_' + str(k)][j] = df[key + '_hlc3'][j]*k + df[key + '_EMA_' + str(i) + '_' + str(k)][j-1]*(1-k)

#EMAs_new
def compute_EMA(args):
    key, df, i, k = args
    EMA_list = []
    for j in range(len(df[key + '_hlc3'])):
        if (j+1) < i:
            EMA = 0
        elif (j+1) == i:
            EMA = (df[key + '_hlc3'][j-i+1:j+1].sum())/i
        else:
            mult = k/(1+i)
            EMA = df[key + '_hlc3'][j]*mult + EMA_list[j-1]*(1-mult)
        EMA_list.append(EMA)

    return key, i, k, EMA_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_EMA, [(key, df, i, k) for key, df in dict_vol_dfs.items() for i in EMA_len_list for k in EMA_smooth_list])

    for key, i, k, EMA_list in results:
        dict_vol_dfs[key][f'{key}_EMA_{i}_{k}'] = EMA_list
        print(key + '_EMA_' + str(i) + '_' + str(k))

# #MACD_old
# for key, df in dict_vol_dfs.items():
#     for i in SMA_len_list:
#         for k in SMA_len_list:
#             if k > i:
#                 print(key + '_MACD_' + str(k) + '_' + str(i))
#                 df[key + '_MACD_' + str(k) + '_' + str(i)] = 0
#                 for j in range(len(df[key + '_hlc3'])):
#                     if df[key + '_EMA_' + str(i)][j] == 0 or df[key + '_EMA_' + str(k)][j] == 0:
#                         df[key + '_MACD_' + str(k) + '_' + str(i)][j] = 0
#                     else:
#                         df[key + '_MACD_' + str(k) + '_' + str(i)][j] = df[key + '_EMA_' + str(k)][j] - df[key + '_EMA_' + str(i)][j]
#             else:
#                 pass

#MACD_new
def compute_MACD(args):
    key, df, i, k, l = args
    MACD_list = []
    if  k > l:
        for j in range(len(df[key + '_hlc3'])):
            if df[key + '_EMA_' + str(k) + '_' + str(i)][j] == 0 or df[key + '_EMA_' + str(l) + '_' + str(i)][j] == 0:
                MACD = 0
            else:
                MACD = df[key + '_EMA_' + str(k) + '_' + str(i)][j] - df[key + '_EMA_' + str(l) + '_' + str(i)][j]
            MACD_list.append(MACD)
    else:
        pass

    return key, i, k, l, MACD_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_MACD, [(key, df, i, k, l) for key, df in dict_vol_dfs.items() for i in EMA_smooth_list for k in EMA_len_list for l in EMA_len_list]) ## yes, EMA_len_list should be iterated through twice

    for key, i, k, l, MACD_list in results:
        if  k > l:
            dict_vol_dfs[key][f'{key}_MACD_{k}_{i}_{l}_{i}'] = MACD_list
            print(key + '_MACD_' + str(k) + '_' + str(i) + '_' + str(l) + '_' + str(i))
        else:
            pass

# # #RSI
# # for key, df in dict_vol_dfs.items():
# #     for i in SMA_len_list:
# #         print(key + '_RSI_' + str(i)) 
# #         df[key + '_RSI_gain_' + str(i)] = 0
# #         df[key + '_RSI_loss_' + str(i)] = 0
# #         df[key + '_avg_RSI_gain_' + str(i)] = 0
# #         df[key + '_avg_RSI_loss_' + str(i)] = 0
# #         df[key + '_RS_' + str(i)] = 0
# #         df[key + '_RSI_' + str(i)] = 0
# #         for k in range(len(df[key + '_hlc3'])):
# #             if k == 0:
# #                 df[key + '_RSI_gain_' + str(i)][k] = 0
# #                 df[key + '_RSI_loss_' + str(i)][k] = 0
# #                 df[key + '_avg_RSI_gain_' + str(i)][k] = 0
# #                 df[key + '_avg_RSI_loss_' + str(i)][k] = 0
# #             else:
# #                 if df[key + '_hlc3'][k] > df[key + '_hlc3'][k-1]:
# #                     df[key + '_RSI_gain_' + str(i)][k] = df[key + '_hlc3'][k] - df[key + '_hlc3'][k-1]
# #                     df[key + '_RSI_loss_' + str(i)][k] = 0
# #                 elif df[key + '_hlc3'][k] < df[key + '_hlc3'][k-1]:
# #                     df[key + '_RSI_loss_' + str(i)][k] = df[key + '_hlc3'][k-1] - df[key + '_hlc3'][k]
# #                     df[key + '_RSI_gain_' + str(i)][k] = 0
# #                 else:
# #                     df[key + '_RSI_gain_' + str(i)][k] = 0
# #                     df[key + '_RSI_loss_' + str(i)][k] = 0
                
# #                 if k < i:
# #                     df[key + '_avg_RSI_gain_' + str(i)][k] = 0
# #                     df[key + '_avg_RSI_loss_' + str(i)][k] = 0
# #                 elif k == i:
# #                     df[key + '_avg_RSI_gain_' + str(i)][k] = df[key + '_RSI_gain_' + str(i)][k-i+1:k+1].sum()/i
# #                     df[key + '_avg_RSI_loss_' + str(i)][k] = df[key + '_RSI_loss_' + str(i)][k-i+1:k+1].sum()/i
# #                 else:
# #                     df[key + '_avg_RSI_gain_' + str(i)][k] = (df[key + '_avg_RSI_gain_' + str(i)][k-1]*(i-1)+df[key + '_RSI_gain_' + str(i)][k])/i
# #                     df[key + '_avg_RSI_loss_' + str(i)][k] = (df[key + '_avg_RSI_loss_' + str(i)][k-1]*(i-1)+df[key + '_RSI_loss_' + str(i)][k])/i
# #             if k >= i:
# #                 df[key + '_RS_' + str(i)][k] = df[key + '_avg_RSI_gain_' + str(i)][k] / df[key + '_avg_RSI_loss_' + str(i)][k]
# #                 df[key + '_RSI_' + str(i)][k] = 100 - (100/(1+df[key + '_RS_' + str(i)][k]))
# #             else:
# #                 df[key + '_RS_' + str(i)][k] = 0
# #                 df[key + '_RSI_' + str(i)][k] = 0
        
# #         df.drop(key + '_RSI_gain_' + str(i), axis=1, inplace=True)
# #         df.drop(key + '_RSI_loss_' + str(i), axis=1, inplace=True)
# #         df.drop(key + '_avg_RSI_gain_' + str(i), axis=1, inplace=True)
# #         df.drop(key + '_avg_RSI_loss_' + str(i), axis=1, inplace=True)
# #         df.drop(key + '_RS_' + str(i), axis=1, inplace=True)

#RSI_new
def compute_RSI(args):
    key, df, i = args
    RSI_gain_list = []
    RSI_loss_list = []
    RSI_avg_gain_list = []
    RSI_avg_loss_list = []
    RS_list = []
    RSI_list = []
    for k in range(len(df[key + '_hlc3'])):
        if k == 0:
            RSI_gain = 0
            RSI_loss = 0
            RSI_avg_gain = 0
            RSI_avg_loss = 0
            RSI_gain_list.append(RSI_gain)
            RSI_loss_list.append(RSI_loss)
            RSI_avg_gain_list.append(RSI_avg_gain)
            RSI_avg_loss_list.append(RSI_avg_loss)
        else:
            if df[key + '_hlc3'][k] > df[key + '_hlc3'][k-1]:
                RSI_gain = df[key + '_hlc3'][k] - df[key + '_hlc3'][k-1]
                RSI_loss = 0
            elif df[key + '_hlc3'][k] < df[key + '_hlc3'][k-1]:
                RSI_loss = df[key + '_hlc3'][k-1] - df[key + '_hlc3'][k]
                RSI_gain = 0
            else:
                RSI_gain = 0
                RSI_loss = 0
            
            RSI_gain_list.append(RSI_gain)
            RSI_loss_list.append(RSI_loss)
                
            if k < i:
                RSI_avg_gain = 0
                RSI_avg_loss = 0
            elif k == i:
                RSI_avg_gain = sum(RSI_gain_list[k-i+1:k+1])/i
                RSI_avg_loss = sum(RSI_loss_list[k-i+1:k+1])/i
            else:
                RSI_avg_gain = (RSI_avg_gain_list[k-1]*(i-1)+RSI_gain_list[k])/i
                RSI_avg_loss = (RSI_avg_loss_list[k-1]*(i-1)+RSI_loss_list[k])/i
            
            RSI_avg_gain_list.append(RSI_avg_gain)
            RSI_avg_loss_list.append(RSI_avg_loss)
            
        if k >= i:
            RS = RSI_avg_gain_list[k] / RSI_avg_loss_list[k]
            RS_list.append(RS)
            RSI = 100 - (100/(1+RS_list[k]))
            RSI_list.append(RSI)
        else:
            RS = 0
            RS_list.append(RS)
            RSI = 0
            RSI_list.append(RSI)
    return key, i, RSI_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_RSI, [(key, df, i) for key, df in dict_vol_dfs.items() for i in SMA_len_list])

    for key, i, RSI_list in results:
        dict_vol_dfs[key][f'{key}_RSI_{i}'] = RSI_list
        print(key + '_RSI_' + str(i))


# #VWAP_old
# for key, df in dict_vol_dfs.items():
#     for i in SMA_len_list:
#         print(key + '_VWAP_' + str(i))
#         df[key + '_VWAP_' + str(i)] = 0
#         for k in range(len(df[key + '_hlc3'])):
#             if k < i:
#                 df[key + '_VWAP_' + str(i)][k] = 0
#             else:
#                 temp_list_1 = df[key + '_hlc3'][k-i+1:k+1].tolist()
#                 temp_list_2 = df[key + '_data_volume'][k-i+1:k+1].tolist()
#                 sum_product = 0
#                 vol_sum = 0
#                 for x, y in zip(temp_list_1, temp_list_2):
#                     sum_product += x * y
#                     vol_sum += y
#                 if df[key + '_SMA_' + str(i)][k] == 0:
#                     df[key + '_VWAP_' + str(i)][k] = 0
#                 elif vol_sum == 0:
#                     df[key + '_VWAP_' + str(i)][k] = df[key + '_data_close'][k]
#                 else:
#                     df[key + '_VWAP_' + str(i)][k] = sum_product/vol_sum
#                 temp_list_1 = []
#                 temp_list_2 = []
#                 sum_product = 0
#                 vol_sum = 0

#VWAP_new and VWAP std dev (non-rolling)
bessel_correction = True
def compute_VWAP(args):
    key, df, i = args
    
    VWAP_list = []
    for k in range(len(df[key + '_hlc3'])):
        if k < i:
            VWAP = 0
            VWAP_std_dev = 0
        else:
            temp_list_1 = df[key + '_hlc3'][k-i+1:k+1].tolist()
            temp_list_2 = df[key + '_volume'][k-i+1:k+1].tolist()
            sum_product = 0
            vol_sum = 0
            for x, y in zip(temp_list_1, temp_list_2):
                sum_product += x * y
                vol_sum += y
            if df[key + '_SMA_' + str(i)][k] == 0:
                VWAP = 0
            elif vol_sum == 0:
                VWAP = df[key + '_close'][k]
            else:
                VWAP = sum_product/vol_sum
            temp_list_1 = []
            temp_list_2 = []
            sum_product = 0
            vol_sum = 0
        VWAP_list.append(VWAP)
    return key, i, VWAP_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_VWAP, [(key, df, i) for key, df in dict_vol_dfs.items() for i in SMA_len_list])

    for key, i, VWAP_list in results:
        dict_vol_dfs[key][f'{key}_VWAP_{i}'] = VWAP_list
        print(key + '_VWAP_' + str(i))

# # #VWAP std dev_old -- need to make 2, one for rolling and one to start the day. Also, think about z-score
# # for key, df in dict_vol_dfs.items():
# #     for i in SMA_len_list:
# #         print(key + '_VWAP_std_dev' + str(i))
# #         df[key + '_VWAP_std_dev' + str(i)] = 0
# #         for k in range(len(df[key + '_hlc3'])):
# #             if k < (i*2-1):
# #                 pass
# #             else:
# #                 x_less_mean_sqrd_list = []
# #                 x_mean = (df[key + '_VWAP_' + str(i)][k-i+1:k+1].sum())/i #note - the first 10 vwap values are 0
# #                 for j in range(i):
# #                     x_less_mean_sqrd_list.append(((df[key + '_VWAP_' + str(i)][k-j]) - x_mean)**2)
# #                 std_dev = (sum(x_less_mean_sqrd_list)/(i-1))**(1/2)
# #                 df[key + '_VWAP_std_dev' + str(i)][k] = std_dev
# #         print(key + '_VWAP_std_dev' + str(i) + '_DONE')


#VWAP std dev_old_2
# def compute_std_dev(args):
#     key, df, i = args
#     std_dev_list = []
#     for k in range(len(df[key + '_hlc3'])):
#         if k < (i*2-1):
#             std_dev = 0
#         else:
#             x_less_mean_sqrd_list = []
#             x_mean = (df[key + '_VWAP_' + str(i)][k-i+1:k+1].sum())/i
#             for j in range(i):
#                 x_less_mean_sqrd_list.append(((df[key + '_VWAP_' + str(i)][k-j]) - x_mean)**2)
#             std_dev = (sum(x_less_mean_sqrd_list)/(i-1))**(1/2)
#         std_dev_list.append(std_dev)
#     return key, i, std_dev_list

# if __name__ == "__main__":
#     # Set the number of processes to utilize
#     num_processes = 16  # Example: Use 4 processes
#     with Pool(num_processes) as pool:
#         results = pool.map(compute_std_dev, [(key, df, i) for key, df in dict_vol_dfs.items() for i in SMA_len_list])

#     for key, i, std_dev_list in results:
#         dict_vol_dfs[key][f'{key}_VWAP_std_dev{i}'] = std_dev_list




#VWAP std dev_new
def compute_std_dev(args):
    key, df, i = args
    std_dev_list = []
    for k in range(len(df[key + '_hlc3'])):
        if k < i:
            std_dev = 0
        else:
            x_less_mean_sqrd_list = []
            x_mean = (df[key + '_VWAP_' + str(i)][k-i+1:k+1].sum())/i
            for j in range(i):
                x_less_mean_sqrd_list .append(((df[key + '_hlc3'][k-j] - vwap_daily)**2)*df[key + '_volume'][k-j])
            std_dev_daily = (sum(x_less_mean_sqrd_daily_list)/(vol_sum))**(1/2)
            z_score_daily = (price - vwap_daily)/std_dev_daily
            if min == 1:
                z_score_daily_list.append(0)
            else:
                z_score_daily_list.append(z_score_daily)
            
            vwap_sum = 0
            vol_sum = 0
            price = df[key + '_hlc3'][k]
            vol = df[key + '_volume'][k]
    return key, i, std_dev_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_std_dev, [(key, df, i) for key, df in dict_vol_dfs.items() for i in SMA_len_list])

    for key, i, std_dev_list in results:
        dict_vol_dfs[key][f'{key}_VWAP_std_dev{i}'] = std_dev_list
    
    
    
#VWAP and VWAP std dev -- daily rolling
def compute_VWAP_daily(args):
    key, df, = args
    VWAP_daily_list = []
    std_dev_daily_list = []
    z_score_daily_list = []
    for k in range(len(df[key + '_hlc3'])):
        min = df['minute'][k]
        if min == 1:
            vwap_sum = 0
            vol_sum = 0
        

        price = df[key + '_hlc3'][k]
        vol = df[key + '_volume'][k]

        vwap_sum += price*vol
        vol_sum += vol
        vwap_daily = vwap_sum/vol_sum
        VWAP_daily_list.append(vwap_daily)

        ### BSH ORIGINAL CALC
        # x_less_mean_sqrd_daily_list = []
        # x_mean_daily = sum(VWAP_daily_list[k-min+1:k+1])/min
        # for j in range(min):
        #     x_less_mean_sqrd_daily_list.append((VWAP_daily_list[k-j] - x_mean_daily)**2)
        # print((sum(x_less_mean_sqrd_daily_list)/(min))**(1/2))
        # std_dev_daily = (sum(x_less_mean_sqrd_daily_list)/(min))**(1/2)
        # std_dev_daily_list.append(std_dev_daily)
        # print(std_dev_daily_list)

        ### TOS CALC
        # if min == 1:
        #     volumeVwapSum = vol * vwap_daily
        #     volumeVwap2Sum = vol * ((vwap_daily)**(2))
        # else:
        #     volumeVwapSum = volumeVwapSum + vol * vwap_daily
        #     volumeVwap2Sum = volumeVwap2Sum + vol * ((vwap_daily)**(2))
        
        # price = volumeVwapSum / vol_sum
        # std_dev_daily =  (max(volumeVwap2Sum/vol_sum - (price**2), 0))**(1/2)
        # std_dev_daily_list.append(std_dev_daily)
        # print(vwap_daily)
        # print(std_dev_daily_list)

        ## NEW CALC from Ben/Marshall Work (matches Yahoo)
        x_less_mean_sqrd_daily_list = []
        # non_zero_weight_count = 0 #
        for j in range(min):
            x_less_mean_sqrd_daily_list.append(((df[key + '_hlc3'][k-j] - vwap_daily)**2)*df[key + '_volume'][k-j])
        std_dev_daily = (sum(x_less_mean_sqrd_daily_list)/(vol_sum))**(1/2)
        z_score_daily = (price - vwap_daily)/std_dev_daily
        if min == 1:
            z_score_daily_list.append(0)
        else:
            z_score_daily_list.append(z_score_daily)


    return key, VWAP_daily_list, z_score_daily_list

if __name__ == "__main__":
    # Set the number of processes to utilize
    num_processes = 16  # Example: Use 4 processes
    with Pool(num_processes) as pool:
        results = pool.map(compute_VWAP_daily, [(key, df) for key, df in dict_vol_dfs.items()])

    for key, VWAP_daily_list, z_score_daily_list in results:
        dict_vol_dfs[key][f'{key}_VWAP_daily'] = VWAP_daily_list
        print(key + '_VWAP_daily')
        dict_vol_dfs[key][f'{key}_z_score_daily'] = z_score_daily_list
        print(key + '_z_score_daily')


AAPL_hlc3
AAPL_candle_size
DIA_hlc3
DIA_candle_size
AMZN_hlc3
AMZN_candle_size
AAPL_candle_color
DIA_candle_color
AMZN_candle_color
AAPL_SMA_10
AAPL_SMA_30
AAPL_SMA_60
AAPL_SMA_120
AAPL_SMA_180
DIA_SMA_10
DIA_SMA_30
DIA_SMA_60
DIA_SMA_120
DIA_SMA_180
AMZN_SMA_10
AMZN_SMA_30
AMZN_SMA_60
AMZN_SMA_120
AMZN_SMA_180
AAPL_EMA_5_1
AAPL_EMA_5_1.5
AAPL_EMA_5_2
AAPL_EMA_5_2.5
AAPL_EMA_5_3
AAPL_EMA_10_1
AAPL_EMA_10_1.5
AAPL_EMA_10_2
AAPL_EMA_10_2.5
AAPL_EMA_10_3
AAPL_EMA_15_1
AAPL_EMA_15_1.5
AAPL_EMA_15_2
AAPL_EMA_15_2.5
AAPL_EMA_15_3
AAPL_EMA_20_1
AAPL_EMA_20_1.5
AAPL_EMA_20_2
AAPL_EMA_20_2.5
AAPL_EMA_20_3
AAPL_EMA_30_1
AAPL_EMA_30_1.5
AAPL_EMA_30_2
AAPL_EMA_30_2.5
AAPL_EMA_30_3
AAPL_EMA_60_1
AAPL_EMA_60_1.5
AAPL_EMA_60_2
AAPL_EMA_60_2.5
AAPL_EMA_60_3
DIA_EMA_5_1
DIA_EMA_5_1.5
DIA_EMA_5_2
DIA_EMA_5_2.5
DIA_EMA_5_3
DIA_EMA_10_1
DIA_EMA_10_1.5
DIA_EMA_10_2
DIA_EMA_10_2.5
DIA_EMA_10_3
DIA_EMA_15_1
DIA_EMA_15_1.5
DIA_EMA_15_2
DIA_EMA_15_2.5
DIA_EMA_15_3
DIA_EMA_20_1
DIA_EMA_20_1.5
DIA_EMA_20_2
DIA_E

In [17]:
for key, dfs in dict_vol_dfs.items():
    # dfs.to_csv('/home/ben_heller/workspace/project_plutus/outputs/out_csv_' + key + '_check2.csv')
    dfs.head((391*10)).to_csv('/home/ben_heller/workspace/project_plutus/outputs/out_csv_' + key + '_check3.csv')
    dfs.to_hdf('/home/ben_heller/workspace/project_plutus/outputs/out_csv_' + key + '_check2.hf', key = key, mode ='w')

In [18]:
### merge dfs for both vol and no vol
merged_df = pd.DataFrame()
for key, df in dict_vol_dfs.items():
  if merged_df.empty:
    merged_df = df
  else:
    merged_df = pd.merge(merged_df, df, on=['timestamp', 'date', 'time', 'minute', 'time_context'] , how='outer').copy()

for key, df in dict_no_vol_dfs.items():
    merged_df = pd.merge(merged_df, df, on=['timestamp', 'date', 'time', 'minute', 'time_context'] , how='outer').copy()

merged_df.head((391*10)).to_csv('/home/ben_heller/workspace/project_plutus/outputs/out_csv_merged_check.csv')

In [19]:

# days = 10
# mins_per_day = 391
# window_size = days*mins_per_day
# roll_df_list = []
# print(len(merged_df))
# print(len(merged_df)-window_size+1)
# for i in range(len(merged_df)-window_size+1):
#     if i <= 2:
#         print(i)
#         df_temp = merged_df.iloc[i:i+window_size]
#         roll_df_list.append(df_temp)
#     else:
#         print(i)

# rolling_merged_df = pd.concat(roll_df_list[:5000], ignore_index=True)
# rolling_merged_df.to_csv('/home/ben_heller/workspace/project_plutus/outputs/out_rolling_merged_check.csv')

# print(df_temp.shape)
# print(len(roll_df_list))


# batch_size = 100
# concatenated_dfs = []
# master_counter = 0
# counter = 0
# file_counter = 0
# for i in range(0, len(roll_df_list), batch_size):
#     counter += batch_size
#     master_counter += batch_size
#     print(master_counter)
#     batch_dfs = roll_df_list[i:i+batch_size]
#     concatenated_df = pd.concat(batch_dfs, ignore_index=True)
#     concatenated_dfs.append(concatenated_df)
#     if counter == 1000:
#         counter = 0
#         file_counter += 1
#         rolling_merged_df = pd.concat(concatenated_dfs, ignore_index=True)
#         rolling_merged_df.to_hdf('/home/ben_heller/workspace/project_plutus/outputs/out_csv_rolling_merged_df_' + str(file_counter) + '.hf', key = key, mode ='w')
#         concatenated_dfs = []
#         del rolling_merged_df







In [20]:
# rolling_merged_df.to_hdf('/home/ben_heller/workspace/project_plutus/outputs/out_csv_rolling_merged_df_dhf5', key = key, mode ='w')

In [21]:
#VWAP std dev_new based on new daily vwap std dev. May need to account for sample - (m-1)/m
# add other features (z-score for high, low, close, and hlc3, as well as Bollinger Bands and scores for bollinger bands)
# manual check of calcs
# dict_no_vol_dfs feature creation 
# rel/scale data (for both price and vol)

# y variable = in next 5 mins, stock should go up 0.5%, but also not go down 0.25% - we need to think on this

# create train/test set
# build model
# train and test
